<a href="https://colab.research.google.com/github/GeorgeTelles/Strategy_Max_Min/blob/main/strategy_max_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>

# Strategy Backtesting: Max and Min

## Description

In financial markets, various strategies are employed to determine buy and sell points through technical analysis of an asset's price chart. One such strategy is the Maxima and Minima strategy, which is the focus of this project.

The Maxima and Minima strategy is based on setting the buy price at the lowest low of the previous two days and the sell price at the highest high of the previous two days. This approach aims to identify potential entry and exit points for trading based on historical price data.

## Project Features

- **Data Collection**: Retrieve historical price data for financial assets (stocks, currencies, etc.).
  
- **Data Modeling and Column Creation**: Create columns representing the two most recent highs and lows for each candle, then determine entry and exit points based on these values. The buy price (entry point) is set at the lowest low of the previous two days, while the sell price (exit point) is set at the highest high of the previous two days.

- **Buy and Sell Price Definition**: Calculate exact buy and sell prices based on the conditions established by the strategy.

- **Backtesting**: Simulate the strategy on historical data to evaluate its performance. This includes tracking ongoing operations, calculating profits and losses, and determining key performance metrics.

- **Profit Calculation and Analysis**: Assess the total profit, percentage gains and losses, and the performance of the holding strategy compared to the strategy.

- **Visualization**: Generate a capital curve plot to visualize the evolution of capital over time and assess the effectiveness of the strategy.

## Technologies Used

- **Language**: Python

- **Libraries**:
  - `pandas` for data manipulation
  - `numpy` for numerical calculations
  - `matplotlib` for visualization
  - `yfinance` for financial data retrieval

- **Development Environment**: Jupyter Notebook or your preferred IDE

## Project Structure

1. **Importing Libraries**: Import necessary libraries for data manipulation, calculations, and visualization.
   
2. **Obtaining Data**: Download historical price data for the chosen asset.

3. **Data Modeling**: Create columns for the two most recent highs and lows, calculate entry and exit points, and define exact buy and sell prices.

4. **Backtesting**: Simulate the strategy, track ongoing operations, and evaluate performance metrics such as total profit, average length of operations, and percentage of gains versus losses.

5. **Results Analysis**: Analyze the performance of the strategy, including total and percentage profits, average operation durations, and comparison with a holding strategy.

6. **Visualization**: Plot the capital evolution over time to visualize the results of the backtesting.

## Results

The results include detailed metrics on the number of operations, percentage of gains and losses, total profit, and a comparison with a holding strategy. The capital evolution plot provides a visual representation of the strategy’s performance over time.



#1. Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import math
import warnings
warnings.filterwarnings('ignore')

#2. Obtendo dados

In [ ]:
ticker = 'ITUB4.SA'
start = '2020-01-01'
df = yf.download(ticker, start)[["Open", "High", "Low", "Close", "Adj Close"]]
df

#3. Modelando os dados e criando colunas

**Vamos criar as colunas que representam as 2 ultimas maximas e minimas de cada candle, e depois criar os pontos de entrada e saida de acordo com a maior maxima ou menor minima, depois Definindo o preço de compra (Ponto de entrada) e o preço de venda (Ponto de saida)**

In [ ]:
df["Target1"] = df["High"].shift(1)
df["Target2"] = df["High"].shift(2)
df["Target"] = df[["Target1", "Target2"]].max(axis=1)

# We don't need them anymore
df.drop(columns=["Target1", "Target2"], inplace=True)

df["Entry1"] = df["Low"].shift(1)
df["Entry2"] = df["Low"].shift(2)
df["Entry"] = df[["Entry1", "Entry2"]].min(axis=1)

# We don't need them anymore
df.drop(columns=["Entry1", "Entry2"], inplace=True)

# Define exact buy price
df['Buy Price'] = np.where(
    df["Low"] < df['Entry'],
    np.where((df['Open'] < df['Entry']), df['Open'], df['Entry']),
    np.nan)

# Define exact sell price
df['Sell Price'] = np.where(
    df["High"] > df['Target'],
    np.where(df['Open'] > df['Target'], df['Open'], df['Target']),
    np.nan)


**Função para calcular o lote da ação**

In [ ]:
# Create a function to round any number to the smalles multiple of 100
def round_down(x):
  return int(math.floor(x / 100.0)) * 100

**Definindo a coluna que vai informar se estamos em um operação ou não: ongoing**

In [ ]:
# Define control variable for ongoing operations
ongoing = False

for i in range(len(df)):

    # If there is an ongoing operation, check if we reached the target price
    if ongoing:
        if not np.isnan(df.iloc[i]['Sell Price']):
            exit = df.iloc[i]['Sell Price']
            ongoing = False

    # If there is no ongoing operation, check if the entry price was achieved
    else:
        if not np.isnan(df.iloc[i]['Buy Price']):
            entry = df.iloc[i]['Buy Price']
            ongoing = True



#4. Backtesting

In [ ]:
# Define backtest parameters
initial_capital = 10000
max_days = 3 # add stop in time

# Control variables
total_capital = [initial_capital] # list with the total capital after every operation
all_profits = [] # list with profits for every operation
all_percent_profits = []
all_buy_dates = []  # Lista para armazenar as datas de compra
all_sell_dates = []  # Lista para armazenar as datas de venda
days_in_operation = 0
gains_total_days = 0
gains_total_operations = 0
losses_total_days = 0
losses_total_operations = 0
ongoing = False

In [ ]:
for i in range(len(df)):
    if ongoing:
        days_in_operation += 1

        # If any of the following conditions are met, the operation will end
        if days_in_operation == max_days or not np.isnan(df.iloc[i]['Sell Price']):
            # Define exit point and total profit
            exit = df.iloc[i]['Sell Price'] if not np.isnan(df.iloc[i]['Sell Price']) else df.iloc[i]['Close']
            profit = shares * (exit - entry)
            percent_profit = (exit - entry) / entry * 100  # Calculate percentage profit

            # Append profit and percent profit to lists
            all_profits.append(profit)
            all_percent_profits.append(percent_profit)
            current_capital = total_capital[-1]
            total_capital.append(current_capital + profit)

            # Store the dates of the buy and sell operations
            all_sell_dates.append(df.index[i])
            if all_buy_dates:
                buy_date = all_buy_dates.pop()  # Remove the most recent buy date
                # print(f"Operation: Buy on {buy_date} at {entry:.2f}, Sell on {df.index[i]} at {exit:.2f}, Profit: ${profit:.2f}, Percent Profit: {percent_profit:.2f}%")

            # Update gains and losses totals
            if profit > 0:
                gains_total_days += days_in_operation
                gains_total_operations += 1
            else:
                losses_total_days += days_in_operation
                losses_total_operations += 1

            ongoing = False
    else:
        if not np.isnan(df.iloc[i]['Buy Price']):
            entry = df.iloc[i]['Buy Price']
            shares = round_down(initial_capital / entry)
            days_in_operation = 0
            ongoing = True

            # Store the date of the buy operation
            all_buy_dates.append(df.index[i])

**Função para calcular os lucros**

In [ ]:
def strategy_test(all_profits):
  num_operations = len(all_profits)
  gains = sum(x >= 0 for x in all_profits)
  pct_gains = 100 * (gains / num_operations)
  losses = num_operations - gains
  pct_losses = 100 - pct_gains
  hold_resul, percent_hold_resul = df['Adj Close'].iloc[-1] - df['Adj Close'].iloc[0], (df['Adj Close'].iloc[-1] - df['Adj Close'].iloc[0]) / df['Adj Close'].iloc[0] * 100
  total_days = gains_total_days + losses_total_days
  total_operations = gains_total_operations + losses_total_operations

  print("Number of operations =", num_operations)
  print("Number of gains =", gains, "or", pct_gains.round(), "%")
  print("Number of loss =", losses, "or", pct_losses.round(), "%")
  print("The total profit was = R$", sum(all_profits), "or", sum(all_percent_profits),"%")
  print(f"O resultado Holding foi de {percent_hold_resul}%")
  print("Average length of operations (in days)", total_days / total_operations)
  print("Average length of gains (in days)", gains_total_days / gains_total_operations)
  print("Average length of losses (in days)", losses_total_days / losses_total_operations)

#5. Resultados

In [ ]:
strategy_test(all_profits)

In [ ]:
def capital_plot(total_capital, all_profits):
  all_profits = [0] + all_profits # make sure both lists are the same size
  cap_evolution = pd.DataFrame({'Capital': total_capital, 'Profit': all_profits})
  plt.title("Curva de Capital")
  plt.xlabel("Total Operações")
  cap_evolution['Capital'].plot()
  plt.show()

capital_plot(total_capital, all_profits)